<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/DGRNet%20-%20Last%20Step%20_Comparison%20Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K
from keras.callbacks import Callback
import plotly
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
from numpy import array
from numpy import hstack
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV, MultiTaskLassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt
from sklearn.linear_model import LinearRegression
from keras.models import Model
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

#-----------------------------------------------------GET DATA - X, Predicted Wndow from MLP, and Derived Labels, for the entire dataset that was gennerated through VAE-----------------------------------

#y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_COSW.csv')
#y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/Calculated_label_COSW.csv')
#x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/whole_data.csv')

y_pred = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/predicted_window_V3.csv')
y_true = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/derived_window_label_V3.csv')
x_orig = np.loadtxt(r'/content/drive/MyDrive/PHD/2024/MLPOutput/test_data_V3.csv')

n_features = 6
window_size = int(x_orig.shape[1]/n_features)
x = x_orig.reshape((x_orig.shape[0],window_size,n_features))


#------------------------------------------------------------------------------------------------------

# split a multivariate long sequence into small sequences, and create target value 1 step in right
def split_sequences(sequences, n_steps):
 X, y = list(), list()
 for i in range(len(sequences)):
 # find the end of this pattern
  end_ix = i + n_steps
 # check if we are beyond the dataset
  if end_ix > len(sequences)-1:
    break
 # gather input and output parts of the pattern
  seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
  X.append(seq_x)
  y.append(seq_y)
 return array(X), array(y)

#-----------------------------------------------CASES - pot window values and pick a few fixed window sizes -------------------------------------------

plt.figure(figsize=(15,6))
plt.subplot(1,2,1)
plt.title("Distribution before Transformation", fontsize=15)
sns.histplot(y_true, kde=True, color="red")
plt.subplot(1,2,2)


fx_window1 = 10
fx_window2 = 20
fx_window3 = 30

pred_step = 1

#-------------------------Take each lng sequence - run VAR, get MSE and pass on------------------------------------------------------------------------


def get_mse(sequences,window):

  x1 = sequences
  x_train=x1[:-pred_step]
  x_test=x1[-pred_step:]
  regressor= VAR(x_train)
  results = regressor.fit(window)
  lag_order = results.k_ar #fitted lag this is just info
  print("lag order",lag_order)

  lagged_values = x_train[-30:] #at least equal to window size
  pred = results.forecast(y=lagged_values, steps=pred_step)
  mse = mean_squared_error(x_test, pred)
  return mse

rmse_list1 = list()
rmse_list2 = list()
rmse_list3 = list()

rmse_list_dgr = list()

n_fold = 6


for i in range(x.shape[0]-1) :

#for i in range(1000) :
  x1 = x[i]
  x_train=x1[:-pred_step]
  x_test=x1[-pred_step:]
  regressor= VAR(x_train)
  results1 = regressor.fit(fx_window1)
  results2 = regressor.fit(fx_window2)
  results3 = regressor.fit(fx_window3)
  results4 = regressor.fit(int(y_true[i]))

  lagged_values = x_train[-29:] #at least equal to window size

  pred1 = results1.forecast(y=lagged_values, steps=pred_step)
  pred2 = results2.forecast(y=lagged_values, steps=pred_step)
  pred3 = results3.forecast(y=lagged_values, steps=pred_step)
  pred4 = results4.forecast(y=lagged_values, steps=pred_step)

  mse1 = mean_squared_error(x_test, pred1)
  mse2 = mean_squared_error(x_test, pred2)
  mse3 = mean_squared_error(x_test, pred3)
  mse4 = mean_squared_error(x_test, pred4)
  print("mse1 = ", mse1)
  print("mse2 = ", mse2)
  print("mse3 = ", mse3)
  print("mse4 = ", mse4)
  rmse_list1.append(mse1)
  rmse_list2.append(mse2)
  rmse_list3.append(mse3)
  rmse_list_dgr.append(mse4)



MSE1 = mean(rmse_list1)
MSE2 = mean(rmse_list2)
MSE3 = mean(rmse_list3)
MSE_DGR = mean(rmse_list_dgr)

RMSE1 = sqrt(MSE1)
RMSE2 = sqrt(MSE2)
RMSE3 = sqrt(MSE3)
RMSE_DGR = sqrt(MSE_DGR)

print (RMSE1,RMSE2,RMSE3,RMSE_DGR)
print ("winner", min(RMSE1,RMSE2,RMSE3,RMSE_DGR))





In [ ]:
from google.colab import drive
drive.mount('/content/drive')